### Test tools for the ambient AI agent team experiment

In [1]:
import dotenv
dotenv.load_dotenv()

import os
import requests

from datetime import datetime
import time

from slack_sdk import WebClient

In [2]:
webhook_url = os.getenv("SLACK_WEBHOOK_URL")
channel_id = os.getenv("SLACK_CHANNEL_ID")
client = WebClient(token=os.getenv("SLACK_BOT_TOKEN"))

def send_slack_message(webhook_url, message):
    payload = {"text": message,}
    response = requests.post(webhook_url, json=payload)
    if response.status_code != 200:
        raise ValueError(f"Request to Slack returned an error {response.status_code}: {response.text}")

In [3]:
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
user_message = f"Right now it is {current_time}\nAll good?"

send_slack_message(webhook_url, user_message)

We can send a message to slack with a button to click.

In [4]:
payload = {
    "text": "MESSAGE HERE. HOW TO PROCEED?",
    "blocks" : [
        {
            "type": "actions",
            "elements": [
                {
                    "type": "button",
                    "text": {"type": "plain_text", "text": "Yes"},
                    "value": "yes",
                    "action_id": "button_click_yes"
                },
                {
                    "type": "button",
                    "text": {"type": "plain_text", "text": "No"},
                    "value": "no",
                    "action_id": "button_click_no"
                }
            ]
        }
    ]
}

Send with polling to wait for response.

In [5]:
def send_message_and_get_ts(message):
    response = client.chat_postMessage(channel=channel_id, text=message)
    return response["ts"]

def poll_for_feedback(channel_id, thread_ts, timeout=60):

    start_time = time.time()

    while time.time() - start_time < timeout:

        response = client.conversations_replies(channel=channel_id, ts=thread_ts)
        messages = response.get("messages", [])
        
        for msg in messages:
            # Skip the original message (which has the same ts as thread_ts)
            if msg.get("ts") == thread_ts:
                continue
                
            # If we found any reply message that's not the original, return it
            return msg.get("text")
            
        # No reply found yet, wait and try again
        time.sleep(5)
        
    return None

In [6]:
# Example usage:
message_text = "Hey from another world man!"
thread_ts = send_message_and_get_ts(message_text)
print("Waiting for user feedback...")

user_feedback = poll_for_feedback(channel_id, thread_ts)
print("Final user feedback:", user_feedback)

Waiting for user feedback...
Final user feedback: bruh


#### Putting everything together for 1 Slack Function

In [61]:
def slack_communication(message, require_feedback=False, timeout=60, image_url=None, image_path=None):
    """
    Send a message to Slack and optionally wait for human feedback.
    
    Args:
        message (str): The message to send to Slack
        require_feedback (bool): Whether to wait for human feedback
        timeout (int): Maximum time to wait for feedback in seconds
        image_url (str): Optional URL of an image to attach (for remote images)
        image_path (str): Optional file path to a local image to upload
        
    Returns:
        str: Human feedback if require_feedback=True, otherwise confirmation message
    """
    try:
        thread_ts = None
        
        # Handle local image upload if provided
        if image_path:
            # Upload the file first
            upload_response = client.files_upload_v2(
                channel=channel_id,
                file=image_path,
                initial_comment=message
            )
            # For local images, we need to send a follow-up message to get the thread_ts
            if require_feedback:
                if require_feedback:

                    thread_ts = None #upload_response.get('file', {}).get('timestamp', {})

                    if not thread_ts:
                        # Fallback if we can't get thread_ts from file upload
                        response = client.chat_postMessage(channel=channel_id, text="Respond to the last image here:")
                        
                        thread_ts = response["ts"]

            else:
                return f"Message and local image posted successfully: {message[:50]}..." if len(message) > 50 else message
        
        # Handle remote image URL
        elif image_url:
            # Prepare blocks for message with image URL
            blocks = [
                {
                    "type": "section",
                    "text": {
                        "type": "mrkdwn",
                        "text": message
                    }
                },
                {
                    "type": "image",
                    "image_url": image_url,
                    "alt_text": "Image"
                }
            ]
            
            response = client.chat_postMessage(
                channel=channel_id, 
                text=message,
                blocks=blocks
            )
            if require_feedback:
                thread_ts = response["ts"]

            else:
                return f"Message with remote image posted successfully: {message[:50]}..." if len(message) > 50 else message
        
        # Text-only message
        else:
            response = client.chat_postMessage(
                channel=channel_id, 
                text=message
            )
            if require_feedback:
                thread_ts = response["ts"]
                
            else:
                return f"Message posted successfully: {message[:50]}..." if len(message) > 50 else message
        
        # Wait for feedback if required
        if require_feedback and thread_ts:
            print(f"Message sent, waiting for feedback (timeout: {timeout}s)...")
            feedback = poll_for_feedback(channel_id, thread_ts, timeout)
            
            if feedback:
                return feedback
            else:
                return "No feedback received within the timeout period."
        
    except Exception as e:
        return f"Error sending message: {str(e)}"

In [66]:
result = slack_communication(
    "yoooo man!", 
)

In [67]:
# Example usage with a local image:
feedback = slack_communication(
    "whts yo name man?", 
    require_feedback=True,
)

print(f"Received feedback: {feedback}")

Message sent, waiting for feedback (timeout: 60s)...
Received feedback: top g


In [62]:
# Example usage with a local image:
result = slack_communication(
    "yo check it out", 
    image_path="../../output/social_media_experiment/AI rats overtaking New York City_img.png"
)

In [63]:
# Example with local image and waiting for feedback:
feedback = slack_communication(
    "What do you think of this local image?", 
    require_feedback=True,
    image_path="../../output/social_media_experiment/AI rats overtaking New York City_img.png"
)
print(f"Received feedback: {feedback}")

Message sent, waiting for feedback (timeout: 60s)...
Received feedback: yppppe


In [64]:
# Example usage with a local image:
result = slack_communication(
    "Here's an image from the internet", 
    image_url="https://miro.medium.com/v2/resize:fit:549/1*g-YZo7s0j46lDQfMmQ955A.png"
)
print(result)

Here's an image from the internet


In [65]:
# Example usage with a local image:
feedback = slack_communication(
    "what do you see here bro?", 
    require_feedback=True,
    image_url="https://miro.medium.com/v2/resize:fit:549/1*g-YZo7s0j46lDQfMmQ955A.png"
)

print(f"Received feedback: {feedback}")

Message sent, waiting for feedback (timeout: 60s)...
Received feedback: nice logo man!
